In [3]:
import networkx as nx
import pandas as pd
import random
import torch
import pickle

## Load W Start Time

In [25]:
### import parquet file
unsw_w_st = pd.read_parquet("C:\\Users\\asus\\Documents\\nids-pcap-dataset\\unsw_parquet_used_dataset\\labeled_w_start_time.parquet",)

In [26]:
unsw_w_st.head()

,source_ip,destination_ip,source_port,destination_port,protocol_x,info_message,attack_category,is_malware,source_ip_info,source_port_info
0,149.171.126.18,175.45.176.1,80,4657,HTTP,HTTP/1.1 200 OK [Malformed Packet],NaN,0,149.171.126.18 HTTP/1.1 200 OK [Malformed Packet],80 HTTP/1.1 200 OK [Malformed Packet]
1,175.45.176.1,149.171.126.18,4657,80,HTTP,GET /oKmwKoVbq HTTP/1.1,NaN,0,175.45.176.1 GET /oKmwKoVbq HTTP/1.1,4657 GET /oKmwKoVbq HTTP/1.1
2,175.45.176.3,149.171.126.18,32473,80,HTTP,GET /level/15/exec/-/buffers/assigned/dump HTT...,NaN,0,175.45.176.3 GET /level/15/exec/-/buffers/assi...,32473 GET /level/15/exec/-/buffers/assigned/du...
3,175.45.176.2,149.171.126.19,59386,8020,HTTP,POST /HttpFileUpload/FileUploadController.do H...,NaN,0,175.45.176.2 POST /HttpFileUpload/FileUploadCo...,59386 POST /HttpFileUpload/FileUploadControlle...
4,149.171.126.14,175.45.176.0,80,21873,HTTP,[TCP ACKed unseen segment] HTTP/1.1 200 OK (t...,NaN,0,149.171.126.14 [TCP ACKed unseen segment] HTTP...,80 [TCP ACKed unseen segment] HTTP/1.1 200 OK ...


## Graph Modeling

In [4]:
import pickle

### nD w source_ip_info

In [27]:
nD_unsw_w_st = unsw_w_st.drop_duplicates(subset=['source_ip_info'], keep='first')

In [28]:
nD_unsw_w_st.is_malware.value_counts()

is_malware
0    15673
1       93
Name: count, dtype: int64

#### source ip as edge 

In [ ]:
nD_graph_w_st_ip = nx.Graph()

for info_message in nD_unsw_w_st["info_message"].unique():
    nD_graph_w_st_ip.add_node(info_message)

for (source_ip), group in nD_unsw_w_st.groupby(["source_ip"]):
    for i in range(len(group) - 1):
        from_node = group.iloc[i]["info_message"]
        to_node = group.iloc[i+1]["info_message"]
        if nD_graph_w_st_ip.has_edge(from_node, to_node):
            nD_graph_w_st_ip[from_node][to_node]["weight"] += 1
        else:
            nD_graph_w_st_ip.add_edge(from_node, to_node, weight=1)

In [48]:
# Create a new undirected graph
nD_graph_w_st_ip = nx.Graph()
node_features = []

# Add nodes for each unique info_message
for source_ip_info in nD_unsw_w_st["source_ip_info"].unique():
    # Get the corresponding is_malware value for the info_message
    is_malware = nD_unsw_w_st[nD_unsw_w_st["source_ip_info"] == source_ip_info]["is_malware"].iloc[0]
    # Add the node with the is_malware value as an attribute
    nD_graph_w_st_ip.add_node(source_ip_info, is_malware=is_malware)
    # Assign node feature value based on is_malware
    # node_features.append([float(is_malware)]) 
    node_features.append([float(random.random())]) # Use float conversion if needed

# Add edges for each unique combination of source_ip and source_port
for (source_ip), group in nD_unsw_w_st.groupby(["source_ip"]):
    # Add edges between each pair of adjacent nodes in the group
    for i in range(len(group) - 1):
        from_node = group.iloc[i]["source_ip_info"]
        to_node = group.iloc[i+1]["source_ip_info"]
        # Add the edge with the correct weight
        if nD_graph_w_st_ip.has_edge(from_node, to_node):
            nD_graph_w_st_ip[from_node][to_node]["weight"] += 1
        else:
            nD_graph_w_st_ip.add_edge(from_node, to_node, weight=1)

In [49]:
pickle.dump(nD_graph_w_st_ip, open('graph_modeling/nD_graph_w_st_ip.pickle', 'wb'))

In [50]:
node_features_tensor = torch.tensor(node_features)

In [51]:
torch.save(node_features_tensor, 'graph_modeling/nD_graph_w_st/node_features_ip.pt')

In [52]:
# Get a list of nodes with no edges
isolated_nodes = list(nx.isolates(nD_graph_w_st_ip))

# Print the isolated nodes
print("Isolated nodes:", len(isolated_nodes), "\n")

Isolated nodes: 0 



In [53]:
# get the number of nodes in the graph
print("Number of nodes:", len(nD_graph_w_st_ip.nodes()), "\n")

Number of nodes: 15766 



#### source port as edge

In [ ]:
nD_graph_w_st_port = nx.Graph()
for info_message in nD_unsw_w_st["info_message"].unique():
    nD_graph_w_st_port.add_node(info_message)

for (source_port), group in nD_unsw_w_st.groupby(["source_port"]):
    for i in range(len(group) - 1):
        from_node = group.iloc[i]["info_message"]
        to_node = group.iloc[i+1]["info_message"]
        if nD_graph_w_st_port.has_edge(from_node, to_node):
            nD_graph_w_st_port[from_node][to_node]["weight"] += 1
        else:
            nD_graph_w_st_port.add_edge(from_node, to_node, weight=1)

In [ ]:
# Get a list of nodes with no edges
isolated_nodes = list(nx.isolates(nD_graph_w_st_port))

# Print the isolated nodes
print("Isolated nodes:", len(isolated_nodes), "\n")

In [ ]:
# get the number of nodes in the graph
print("Number of nodes:", len(nD_graph_w_st_port.nodes()), "\n")

In [ ]:
pickle.dump(nD_graph_w_st_port, open('graph_modeling/nD_graph_w_st_port.pickle', 'wb'))

#### source port & source ip as edge

In [ ]:
nD_graph_w_st_ip_port = nx.Graph()

for info_message in nD_unsw_w_st["info_message"].unique():
    nD_graph_w_st_ip_port.add_node(info_message, is_malware=is_malware)

for (source_ip, source_port), group in nD_unsw_w_st.groupby(["source_ip", "source_port"]):
    for i in range(len(group) - 1):
        from_node = group.iloc[i]["info_message"]
        to_node = group.iloc[i+1]["info_message"]
        if nD_graph_w_st_ip_port.has_edge(from_node, to_node):
            nD_graph_w_st_ip_port[from_node][to_node]["weight"] += 1
        else:
            nD_graph_w_st_ip_port.add_edge(from_node, to_node, weight=1)

In [ ]:
# Get a list of nodes with no edges
isolated_nodes = list(nx.isolates(nD_graph_w_st_ip_port))

# Print the isolated nodes
print("Isolated nodes:", len(isolated_nodes), "\n")

In [ ]:
# get the number of nodes in the graph
print("Number of nodes:", len(nD_graph_w_st_ip_port.nodes()), "\n")

In [ ]:
pickle.dump(nD_graph_w_st_port, open('graph_modeling/nD_graph_w_st_ip_port.pickle', 'wb'))

## Load WO Start Time

In [4]:
### import parquet file
unsw_wo_st = pd.read_parquet("C:\\Users\\asus\\Documents\\nids-pcap-dataset\\unsw_parquet_used_dataset\\labeled_wo_start_time.parquet",)

In [5]:
unsw_wo_st.head()

,source_ip,destination_ip,source_port,destination_port,protocol_x,info_message,attack_category,is_malware,source_ip_info,source_port_info
0,149.171.126.18,175.45.176.1,80,4657,HTTP,HTTP/1.1 200 OK [Malformed Packet],NaN,0,149.171.126.18 HTTP/1.1 200 OK [Malformed Packet],80 HTTP/1.1 200 OK [Malformed Packet]
1,175.45.176.1,149.171.126.18,4657,80,HTTP,GET /oKmwKoVbq HTTP/1.1,NaN,0,175.45.176.1 GET /oKmwKoVbq HTTP/1.1,4657 GET /oKmwKoVbq HTTP/1.1
2,175.45.176.3,149.171.126.18,32473,80,HTTP,GET /level/15/exec/-/buffers/assigned/dump HTT...,NaN,0,175.45.176.3 GET /level/15/exec/-/buffers/assi...,32473 GET /level/15/exec/-/buffers/assigned/du...
3,175.45.176.2,149.171.126.19,59386,8020,HTTP,POST /HttpFileUpload/FileUploadController.do H...,NaN,0,175.45.176.2 POST /HttpFileUpload/FileUploadCo...,59386 POST /HttpFileUpload/FileUploadControlle...
4,149.171.126.14,175.45.176.0,80,21873,HTTP,[TCP ACKed unseen segment] HTTP/1.1 200 OK (t...,NaN,0,149.171.126.14 [TCP ACKed unseen segment] HTTP...,80 [TCP ACKed unseen segment] HTTP/1.1 200 OK ...


In [6]:
unsw_wo_st.is_malware.value_counts()

is_malware
0    472933
1     17093
Name: count, dtype: int64

## Graph Modeling Source IP and Info Message

### nD w source_ip_info

In [7]:
nD_unsw_wo_st = unsw_wo_st.drop_duplicates(subset=['source_ip_info'])

In [8]:
nD_unsw_wo_st.is_malware.value_counts()

is_malware
1    10707
0     5059
Name: count, dtype: int64

#### source ip as edge 

In [ ]:
# Create a new undirected graph
nD_graph_wo_st_ip = nx.Graph()

# Add nodes for each unique info_message
for source_ip_info in nD_unsw_wo_st["source_ip_info"].unique():
    # Get the corresponding is_malware value for the info_message
    is_malware = nD_unsw_wo_st[nD_unsw_wo_st["source_ip_info"] == source_ip_info]["is_malware"].iloc[0]
    # Add the node with the is_malware value as an attribute
    nD_graph_wo_st_ip.add_node(source_ip_info, is_malware=is_malware)

# Add edges for each unique combination of source_ip and source_port
for (source_ip), group in nD_unsw_wo_st.groupby(["source_ip"]):
    # Add edges between each pair of adjacent nodes in the group
    for i in range(len(group) - 1):
        from_node = group.iloc[i]["source_ip_info"]
        to_node = group.iloc[i+1]["source_ip_info"]
        # Add the edge with the correct weight
        if nD_graph_wo_st_ip.has_edge(from_node, to_node):
            nD_graph_wo_st_ip[from_node][to_node]["weight"] += 1
        else:
            nD_graph_wo_st_ip.add_edge(from_node, to_node, weight=1)

In [72]:
# Create a new undirected graph
nD_graph_wo_st_ip = nx.Graph()
node_features = []

# Add nodes for each unique info_message
for source_ip_info in nD_unsw_wo_st["source_ip_info"].unique():
    # Get the corresponding is_malware value for the info_message
    is_malware = nD_unsw_wo_st[nD_unsw_wo_st["source_ip_info"] == source_ip_info]["is_malware"].iloc[0]
    # Add the node with the is_malware value as an attribute
    nD_graph_wo_st_ip.add_node(source_ip_info, is_malware=is_malware)
    # Assign node feature value based on is_malware
    # node_features.append([float(is_malware)])
    # node_features.append([float(1.0)])
    node_features.append([float(len(source_ip_info))]) 
    # node_features.append([[float(random.random())]]) # Use float conversion if needed

# Add edges for each unique combination of source_ip and source_port
for (source_ip), group in nD_unsw_wo_st.groupby(["source_ip"]):
    for i in range(len(group) - 1):
        from_node = group.iloc[i]["source_ip_info"]
        to_node = group.iloc[i+1]["source_ip_info"]
        # Add the edge with the correct weight
        if nD_graph_wo_st_ip.has_edge(from_node, to_node):
            nD_graph_wo_st_ip[from_node][to_node]["weight"] += 1
        else:
            nD_graph_wo_st_ip.add_edge(from_node, to_node, weight=1)

In [73]:
import torch

In [74]:
import pickle

In [75]:
node_features_tensor = torch.tensor(node_features)

In [76]:
node_features_tensor = torch.nn.functional.normalize(node_features_tensor, p=4.0, dim=0, eps=1e-12, out=None)

In [63]:
# node_feature_tensor = node_features_tensor.unsqueeze(1)

In [ ]:
# Convert the node_features_tensor to a pandas Series
node_features_series = pd.Series(node_features_tensor.tolist())

# Use the value_counts() method to get the count of unique values
value_counts = node_features_series.value_counts()

# Print the value counts
print(value_counts)

0.089242    15766
Name: count, dtype: int64


In [77]:
torch.save(node_features_tensor, 'graph_modeling/nD_graph_wo_st/node_features_ip.pt')

In [78]:
pickle.dump(nD_graph_wo_st_ip, open('graph_modeling/nD_graph_wo_st/nD_graph_wo_st_ip.pickle', 'wb'))

In [79]:
# Get a list of nodes with no edges
isolated_nodes = list(nx.isolates(nD_graph_wo_st_ip))

# Print the isolated nodes
print("Isolated nodes:", len(isolated_nodes), "\n")

Isolated nodes: 0 



In [80]:
# get the number of nodes in the graph
print("Number of nodes:", len(nD_graph_wo_st_ip.nodes()), "\n")

Number of nodes: 15766 



#### source port as edge 

In [ ]:
# Create a new undirected graph
nD_graph_wo_st_port = nx.Graph()

# Add nodes for each unique source_ip_info
for source_ip_info in nD_unsw_wo_st["source_ip_info"].unique():
    # Get the corresponding is_malware value for the source_ip_info
    is_malware = nD_unsw_wo_st[nD_unsw_wo_st["source_ip_info"] == source_ip_info]["is_malware"].iloc[0]
    # Add the node with the is_malware value as an attribute
    nD_graph_wo_st_port.add_node(source_ip_info, is_malware=is_malware)

# Add edges for each unique combination of source_ip and source_port
for (source_port), group in nD_unsw_wo_st.groupby(["source_port"]):
    # Add edges between each pair of adjacent nodes in the group
    for i in range(len(group) - 1):
        from_node = group.iloc[i]["source_ip_info"]
        to_node = group.iloc[i+1]["source_ip_info"]
        # Add the edge with the correct weight
        if nD_graph_wo_st_port.has_edge(from_node, to_node):
            nD_graph_wo_st_port[from_node][to_node]["weight"] += 1
        else:
            nD_graph_wo_st_port.add_edge(from_node, to_node, weight=1)

In [ ]:
# Get a list of nodes with no edges
isolated_nodes = list(nx.isolates(nD_graph_wo_st_port))

# Print the isolated nodes
print("Isolated nodes:", len(isolated_nodes), "\n")

In [ ]:
# get the number of nodes in the graph
print("Number of nodes:", len(nD_graph_wo_st_port.nodes()), "\n")

In [ ]:
pickle.dump(nD_graph_wo_st_port, open('graph_modeling/nD_graph_wo_st/nD_graph_wo_st_port.pickle', 'wb'))

#### source ip & source port as edge

In [ ]:
# Create a new undirected graph
nD_graph_wo_st_ip_port = nx.Graph()

# Add nodes for each unique source_ip_info
for source_ip_info in nD_unsw_wo_st["source_ip_info"].unique():
    # Get the corresponding is_malware value for the source_ip_info
    is_malware = nD_unsw_wo_st[nD_unsw_wo_st["source_ip_info"] == source_ip_info]["is_malware"].iloc[0]
    # Add the node with the is_malware value as an attribute
    nD_graph_wo_st_ip_port.add_node(source_ip_info, is_malware=is_malware)

# Add edges for each unique combination of source_ip and source_port
for (source_ip, source_port), group in nD_unsw_wo_st.groupby(["source_ip", "source_port"]):
    # Add edges between each pair ofac adjent nodes in the group
    for i in range(len(group) - 1):
        from_node = group.iloc[i]["source_ip_info"]
        to_node = group.iloc[i+1]["source_ip_info"]
        # Add the edge with the correct weight
        if nD_graph_wo_st_ip_port.has_edge(from_node, to_node):
            nD_graph_wo_st_ip_port[from_node][to_node]["weight"] += 1
        else:
            nD_graph_wo_st_ip_port.add_edge(from_node, to_node, weight=1)

In [ ]:
# Get a list of nodes with no edges
isolated_nodes = list(nx.isolates(nD_graph_wo_st_ip_port))

# Print the isolated nodes
print("Isolated nodes:", len(isolated_nodes), "\n")

In [ ]:
# get the number of nodes in the graph
print("Number of nodes:", len(nD_graph_wo_st_port.nodes()), "\n")

In [ ]:
pickle.dump(nD_graph_wo_st_ip_port, open('graph_modeling/nD_graph_wo_st/nD_graph_wo_st_ip_port.pickle', 'wb'))

## Graph Modeling Source Port and Info Message

In [7]:
nD_unsw_spi_wo_st = unsw_wo_st.drop_duplicates(subset=['source_port_info'])

In [8]:
nD_unsw_spi_wo_st.is_malware.value_counts()

is_malware
0    81167
1    15336
Name: count, dtype: int64

#### source ip as edge

In [30]:
# Create a new undirected graph
nD_graph_spi_wo_st_ip = nx.Graph()
node_features = []

# Add nodes for each unique info_message
for source_ip_info in nD_unsw_spi_wo_st["source_port_info"].unique():
    # Get the corresponding is_malware value for the info_message
    is_malware = nD_unsw_spi_wo_st[nD_unsw_spi_wo_st["source_port_info"] == source_ip_info]["is_malware"].iloc[0]
    # Add the node with the is_malware value as an attribute
    nD_graph_spi_wo_st_ip.add_node(source_ip_info, is_malware=is_malware)
    # Assign node feature value based on is_malware
    node_features.append([float(is_malware)])
    # node_features.append([float(1.0)]) 
    # node_features.append([float(random.random())]) # Use float conversion if needed

# Add edges for each unique combination of source_ip and source_port
for (source_ip), group in nD_unsw_spi_wo_st.groupby(["source_ip"]):
    # Add edges between each pair of adjacent nodes in the group
    for i in range(len(group) - 1):
        from_node = group.iloc[i]["source_port_info"]
        to_node = group.iloc[i+1]["source_port_info"]
        # Add the edge with the correct weight
        if nD_graph_spi_wo_st_ip.has_edge(from_node, to_node):
            nD_graph_spi_wo_st_ip[from_node][to_node]["weight"] += 1
        else:
            nD_graph_spi_wo_st_ip.add_edge(from_node, to_node, weight=1)

In [31]:
node_features_tensor = torch.tensor(node_features)

In [32]:
torch.save(node_features_tensor, 'graph_modeling/nD_graph_wo_st/node_features_spi_ip.pt')

In [33]:
pickle.dump(nD_graph_spi_wo_st_ip, open('graph_modeling/nD_graph_wo_st/nD_graph_spi_wo_st_ip.pickle', 'wb'))

In [34]:
# Get a list of nodes with no edges
isolated_nodes = list(nx.isolates(nD_graph_spi_wo_st_ip))

# Print the isolated nodes
print("Isolated nodes:", len(isolated_nodes), "\n")

Isolated nodes: 4 



In [35]:
# get the number of nodes in the graph
print("Number of nodes:", len(nD_graph_spi_wo_st_ip.nodes()), "\n")

Number of nodes: 96503 



## Trial

In [52]:
nD_unsw_wo_st = unsw_wo_st.drop_duplicates(subset=['source_ip_info'])

In [53]:
nD_unsw_wo_st.is_malware.value_counts()

is_malware
1    10707
0     5059
Name: count, dtype: int64

In [54]:
len(nD_unsw_wo_st.source_port_info.unique())

15013

In [33]:
value_counts = nD_unsw_wo_st.is_malware.value_counts()
if value_counts[1] > 4500:
    df_to_lower = nD_unsw_wo_st[nD_unsw_wo_st['is_malware'] == 1].sample(n=4500, random_state=0)
    nD_unsw_wo_st = pd.concat([nD_unsw_wo_st[nD_unsw_wo_st['is_malware'] == 0], df_to_lower])

In [34]:
updated_value_counts = nD_unsw_wo_st['is_malware'].value_counts()
print(updated_value_counts)

is_malware
0    5059
1    4500
Name: count, dtype: int64


In [35]:
nD_unsw_wo_st = nD_unsw_wo_st.drop_duplicates(subset=['source_ip_info'])

In [55]:
## print number of unique source_port_info
print("Number of unique source_ip_info:", len(nD_unsw_wo_st["source_ip_info"].unique()), "\n")

Number of unique source_ip_info: 15766 



In [56]:
# Create a new undirected graph
nD_graph_trial = nx.Graph()
node_features = []

# Add nodes for each unique info_message
for source_ip_info in nD_unsw_wo_st["source_ip_info"].unique():
    # Get the corresponding is_malware value for the info_message
    is_malware = nD_unsw_wo_st[nD_unsw_wo_st["source_ip_info"] == source_ip_info]["is_malware"].iloc[0]
    # Add the node with the is_malware value as an attribute
    nD_graph_trial.add_node(source_ip_info, is_malware=is_malware)
    # Assign node feature value based on is_malware
    node_features.append([float(len(nD_unsw_wo_st.info_message))])
    # node_features.append([float(round(random.random(),4))]) # Use float conversion if needed

# Add edges for each unique combination of source_ip and source_port
for (source_ip), group in nD_unsw_wo_st.groupby(["source_ip"]):
    # Add edges between each pair of adjacent nodes in the group
    for i in range(len(group) - 1):
        from_node = group.iloc[i]["source_ip_info"]
        to_node = group.iloc[i+1]["source_ip_info"]
        # Add the edge with the correct weight
        if nD_graph_trial.has_edge(from_node, to_node):
            nD_graph_trial[from_node][to_node]["weight"] += 1
        else:
            nD_graph_trial.add_edge(from_node, to_node, weight=1)

In [64]:
nx.average_degree_connectivity(nD_graph_trial)

{1: 2.0, 2: 1.9978341189960505}

In [57]:
node_features_tensor = torch.tensor(node_features)

In [58]:
node_features_tensor = torch.nn.functional.normalize(node_features_tensor, p=4.0, dim=0, eps=1e-12, out=None)

In [59]:
torch.save(node_features_tensor, 'graph_modeling/nD_graph_wo_st/node_features_ip_trial.pt')

In [60]:
pickle.dump(nD_graph_trial, open('graph_modeling/nD_graph_wo_st/nD_graph_trial.pickle', 'wb'))

In [61]:
# get the number of nodes in the graph
print("Number of nodes:", len(nD_graph_trial.nodes()), "\n")

Number of nodes: 15766 



In [62]:
label = nD_unsw_wo_st['is_malware'].to_numpy()
label_trial = torch.tensor(label, dtype=torch.float)
# label_trial = torch.tensor(nD_unsw_wo_st['is_malware'].values)

In [63]:
torch.save(label_trial, 'graph_modeling/nD_graph_wo_st/label_trial.pt')
len(label_trial)


15766